In [377]:
%pwd
%cd template
%pwd

In [378]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import set_config
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Constants
data_file_path = "./data/home-data-for-ml-course/train.csv"
test_size = 0.2
val_size = 0.2
random_state = 0
set_config(transform_output="pandas")

# Load data
df = pd.read_csv(data_file_path)

# Target and features
target = "SalePrice"
y = df.SalePrice

# All numeric without missing values
features = list(set(df.columns) - set(["SalePrice"]))
X_orig = df[features]
X = X_orig.copy()

In [379]:
# Numerical vs categorical features
categorical_cols = list(X.select_dtypes(include=["object"]).columns)
numerical_cols = list(set(X.columns) - set(categorical_cols))
area_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea"]

In [380]:
# Handle missing values by replacing with mean or mode
cols_with_missing_vals = X.isnull().sum()
cols_with_missing_vals = cols_with_missing_vals[cols_with_missing_vals > 0].index

cols_with_missing_vals_categorical = list(set(categorical_cols).intersection(set(cols_with_missing_vals)))
cols_with_missing_vals_numerical = list(set(numerical_cols).intersection(set(cols_with_missing_vals)))

# Mask numerical nulls with mean
for col in cols_with_missing_vals_numerical:
    print(f"Replacing nulls in {col} with mean")
    X[col] = X[col].mask(X[col].isnull(), X[col].mean())

# Mask categorical nulls with mode
for col in cols_with_missing_vals_categorical:
    print(f"Replacing nulls in {col} with mode")
    X[col] = X[col].mask(X[col].isnull(), X[col].mode().values[0])

assert np.all(X.isnull().sum() == 0), "Null values are still present in the data"

In [381]:
# Ordinal-encode categorical variables
# pd.options.mode.chained_assignment = None

for col in categorical_cols:
    values, index = pd.factorize(X.loc[:, col])
    # Disable chained assignment for this line
    X[col] = values

# pd.options.mode.chained_assignment = "warn"

In [382]:
# Standardize all numerical features
cols = numerical_cols
X[cols] = (X[cols] - X[cols].mean()) / X[cols].std()

# Feature Importance

In [383]:
# Use mutual information to select features
from sklearn.feature_selection import mutual_info_regression

# Compute mutual information scores
mi_scores = pd.Series(
    data=mutual_info_regression(X[numerical_cols], y, discrete_features=False),
        index=numerical_cols)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

In [370]:
# Plot mutual information scores
import seaborn as sns
from matplotlib import pyplot as plt
plt.figure(figsize=(10,16))
N = 30
sns.barplot(x=mi_scores.values[0:N], y=mi_scores.index[0:N])

# Creating Features

In [371]:
# Create some features
# LotFrontage: Linear feet of street connected to property
# LotArea: Lot size in square feet
# MasVnrArea: Masonry veneer area in square feet
# # BsmtFinSF1: Type 1 finished square feet
# BsmtFinSF2: Type 2 finished square feet
# BsmtUnfSF: Unfinished square feet of basement area
# TotalBsmtSF: Total square feet of basement area
# 1stFlrSF: First Floor square feet
# 2ndFlrSF: Second floor square feet
# LowQualFinSF: Low quality finished square feet (all floors)
# GrLivArea: Above grade (ground) living area square feet
# BsmtFullBath: Basement full bathrooms
# BsmtHalfBath: Basement half bathrooms
# FullBath: Full bathrooms above grade
# HalfBath: Half baths above grade
# Bedroom: Bedrooms above grade (does NOT include basement bedrooms)
# Kitchen: Kitchens above grade
# TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
# Fireplaces: Number of fireplaces
# GarageCars: Size of garage in car capacity
# GarageArea: Size of garage in square feet
# WoodDeckSF: Wood deck area in square feet
# OpenPorchSF: Open porch area in square feet
# EnclosedPorch: Enclosed porch area in square feet
# 3SsnPorch: Three season porch area in square feet
# ScreenPorch: Screen porch area in square feet
# PoolArea: Pool area in square feet
# MiscVal: $Value of miscellaneous feature
cols = ["LotArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","BsmtFullBath","BsmtHalfBath","FullBath","HalfBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal"]
N = len(cols)
for m in range(N):
    for n in range(m + 1, N):
        X[f"{cols[m]}/{cols[n]}"] = (X[cols[m]]/X[cols[n]]).fillna(value=0).replace(np.inf, 0)

# The above transform divides one feature by another. Other possible transforms:
# logs, counts, splitting or combining string features, a grouped feature that
# the sample belongs to

# Cluster Labels as Features

In [372]:
# Use k-means to create cluster labels and use these as a feature
kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto", max_iter=300).fit(X[["GrLivArea","TotalBsmtSF"]])
X["cluster"] = kmeans.predict(X[["GrLivArea","TotalBsmtSF"]])

In [373]:
# Could use the distance to cluster centroids as a feature as well
kmeans.transform(X[["GrLivArea","TotalBsmtSF"]])

# PCA for Feature Engineering

In [374]:
def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

In [389]:
# PCA basics
# Always standardize data for PCA
pca = PCA()
cols = area_cols
X_pca = pca.fit_transform(X[cols]) # the data transformed by PCA; each column is a principal component

# pca.components_; each row is a principal component, each column is a feature
df_pca_comp = pd.DataFrame(data = pca.components_,
                           columns = cols)

# pc0 represent lots with high LotFrontage and high LotArea
# pc1 represent lots with low LotFrontage and high LotArea, eg, narrow lots

# Explained variante ratio
df_ev = pd.DataFrame(data = pca.explained_variance_ratio_,
                     index = df_pca_comp.index,
                     columns = ["ev"])
# Each row is the fraction of variance explained by the corresponding principal component
# pc0 explains 23.4% of the variance

df_ev

In [376]:
plot_variance(pca)

In [390]:
mi_scores = pd.Series(
    data=mutual_info_regression(X_pca, y, discrete_features=False),
        index=X_pca.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

# component 0 has a high mutual information score, so it can be used as a feature as-is


In [396]:
# Sort the absolute value of loadings of component 0
# Loadings that have high absolute values can be used to create new features (products, ratios, etc.)
df_pca_comp.loc[0,:].abs().sort_values(ascending=False)

In [397]:
# Create a Boxen plot (sns.catplot) of the principal components to see outliers

# Target Encoding
Encode categorical variables using a number derived from the target. Known by
other names as mean encoding, bin counting, likelihood encoding, impact
encoding, leave-one-out encoding